In [6]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Hospital Billing System\n",
    "\n",
    "This notebook provides a Python implementation of a hospital billing system with database integration."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sqlite3\n",
    "from datetime import datetime\n",
    "import pandas as pd\n",
    "\n",
    "class HospitalBillingSystem:\n",
    "    def __init__(self):\n",
    "        self.conn = sqlite3.connect(':memory:')\n",
    "        self.create_tables()\n",
    "        \n",
    "    def create_tables(self):\n",
    "        cursor = self.conn.cursor()\n",
    "        \n",
    "        cursor.execute('''\n",
    "        CREATE TABLE patients (\n",
    "            patient_id INTEGER PRIMARY KEY AUTOINCREMENT,\n",
    "            name TEXT NOT NULL,\n",
    "            age INTEGER NOT NULL,\n",
    "            gender TEXT NOT NULL,\n",
    "            contact TEXT NOT NULL,\n",
    "            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP\n",
    "        )''')\n",
    "        \n",
    "        cursor.execute('''\n",
    "        CREATE TABLE bills (\n",
    "            bill_id INTEGER PRIMARY KEY AUTOINCREMENT,\n",
    "            patient_id INTEGER NOT NULL,\n",
    "            bill_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,\n",
    "            subtotal REAL NOT NULL,\n",
    "            tax REAL NOT NULL,\n",
    "            total_amount REAL NOT NULL,\n",
    "            FOREIGN KEY (patient_id) REFERENCES patients(patient_id)\n",
    "        )''')\n",
    "        \n",
    "        cursor.execute('''\n",
    "        CREATE TABLE treatments (\n",
    "            treatment_id INTEGER PRIMARY KEY AUTOINCREMENT,\n",
    "            bill_id INTEGER NOT NULL,\n",
    "            description TEXT NOT NULL,\n",
    "            cost REAL NOT NULL,\n",
    "            FOREIGN KEY (bill_id) REFERENCES bills(bill_id)\n",
    "        )''')\n",
    "        \n",
    "        self.conn.commit()\n",
    "    \n",
    "    def add_patient(self, name, age, gender, contact):\n",
    "        cursor = self.conn.cursor()\n",
    "        cursor.execute('''\n",
    "        INSERT INTO patients (name, age, gender, contact)\n",
    "        VALUES (?, ?, ?, ?)\n",
    "        ''', (name, age, gender, contact))\n",
    "        self.conn.commit()\n",
    "        return cursor.lastrowid\n",
    "    \n",
    "    def create_bill(self, patient_id, treatments):\n",
    "        subtotal = sum(t['cost'] for t in treatments)\n",
    "        tax = subtotal * 0.18\n",
    "        total = subtotal + tax\n",
    "        \n",
    "        cursor = self.conn.cursor()\n",
    "        cursor.execute('''\n",
    "        INSERT INTO bills (patient_id, subtotal, tax, total_amount)\n",
    "        VALUES (?, ?, ?, ?)\n",
    "        ''', (patient_id, subtotal, tax, total))\n",
    "        bill_id = cursor.lastrowid\n",
    "        \n",
    "        for treatment in treatments:\n",
    "            cursor.execute('''\n",
    "            INSERT INTO treatments (bill_id, description, cost)\n",
    "            VALUES (?, ?, ?)\n",
    "            ''', (bill_id, treatment['description'], treatment['cost']))\n",
    "        \n",
    "        self.conn.commit()\n",
    "        return bill_id\n",
    "    \n",
    "    def get_bill_details(self, bill_id):\n",
    "        cursor = self.conn.cursor()\n",
    "        \n",
    "        # Get bill and patient info\n",
    "        cursor.execute('''\n",
    "        SELECT b.*, p.name, p.age, p.gender, p.contact\n",
    "        FROM bills b\n",
    "        JOIN patients p ON b.patient_id = p.patient_id\n",
    "        WHERE b.bill_id = ?\n",
    "        ''', (bill_id,))\n",
    "        bill = cursor.fetchone()\n",
    "        \n",
    "        # Get treatments\n",
    "        cursor.execute('''\n",
    "        SELECT description, cost\n",
    "        FROM treatments\n",
    "        WHERE bill_id = ?\n",
    "        ''', (bill_id,))\n",
    "        treatments = cursor.fetchall()\n",
    "        \n",
    "        return {\n",
    "            'bill': bill,\n",
    "            'treatments': treatments\n",
    "        }\n",
    "    \n",
    "    def display_bill(self, bill_id):\n",
    "        details = self.get_bill_details(bill_id)\n",
    "        bill = details['bill']\n",
    "        treatments = details['treatments']\n",
    "        \n",
    "        print(\"===== Hospital Bill =====\")\n",
    "        print(f\"Bill ID: {bill_id}\")\n",
    "        print(f\"Date: {bill[2]}\")\n",
    "        print(f\"\\nPatient: {bill[5]}\")\n",
    "        print(f\"Age: {bill[6]}\")\n",
    "        print(f\"Gender: {bill[7]}\")\n",
    "        print(f\"Contact: {bill[8]}\")\n",
    "        \n",
    "        print(\"\\nTreatments/Services:\")\n",
    "        for idx, treatment in enumerate(treatments, 1):\n",
    "            print(f\"{idx}. {treatment[0]} - ₹{treatment[1]:.2f}\")\n",
    "        \n",
    "        print(f\"\\nSubtotal: ₹{bill[3]:.2f}\")\n",
    "        print(f\"Tax (18%): ₹{bill[4]:.2f}\")\n",
    "        print(f\"Total: ₹{bill[5]:.2f}\")\n",
    "        print(\"=======================\")\n",
    "    \n",
    "    def export_to_excel(self, bill_id, filename):\n",
    "        details = self.get_bill_details(bill_id)\n",
    "        bill = details['bill']\n",
    "        treatments = details['treatments']\n",
    "        \n",
    "        # Create DataFrames\n",
    "        patient_df = pd.DataFrame({\n",
    "            'Field': ['Patient Name', 'Age', 'Gender', 'Contact'],\n",
    "            'Value': [bill[5], bill[6], bill[7], bill[8]]\n",
    "        })\n",
    "        \n",
    "        treatments_df = pd.DataFrame(treatments, columns=['Description', 'Cost'])\n",
    "        \n",
    "        totals_df = pd.DataFrame({\n",
    "            'Description': ['Subtotal', 'Tax (18%)', 'Total'],\n",
    "            'Amount': [bill[3], bill[4], bill[5]]\n",
    "        })\n",
    "        \n",
    "        # Write to Excel\n",
    "        with pd.ExcelWriter(filename) as writer:\n",
    "            patient_df.to_excel(writer, sheet_name='Patient Info', index=False)\n",
    "            treatments_df.to_excel(writer, sheet_name='Treatments', index=False)\n",
    "            totals_df.to_excel(writer, sheet_name='Totals', index=False)\n",
    "        \n",
    "        print(f\"Bill exported to {filename}\")\n",
    "\n",
    "# Example usage\n",
    "hbs = HospitalBillingSystem()\n",
    "\n",
    "# Add a patient\n",
    "patient_id = hbs.add_patient(\"Shraddha Sharma\", 21, \"F\", \"9921709429\")\n",
    "\n",
    "# Create a bill with treatments\n",
    "treatments = [\n",
    "    {\"description\": \"General Consultation\", \"cost\": 500},\n",
    "    {\"description\": \"Blood Test\", \"cost\": 300}\n",
    "]\n",
    "bill_id = hbs.create_bill(patient_id, treatments)\n",
    "\n",
    "# Display the bill\n",
    "hbs.display_bill(bill_id)\n",
    "\n",
    "# Export to Excel\n",
    "hbs.export_to_excel(bill_id, \"hospital_bill.xlsx\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Interactive version for Jupyter\n",
    "from ipywidgets import interact, interactive, fixed, interact_manual\n",
    "import ipywidgets as widgets\n",
    "from IPython.display import display\n",
    "\n",
    "hbs = HospitalBillingSystem()\n",
    "\n",
    "patient_name = widgets.Text(description=\"Name:\")\n",
    "patient_age = widgets.IntText(description=\"Age:\")\n",
    "patient_gender = widgets.Dropdown(description=\"Gender:\", options=['M', 'F', 'Other'])\n",
    "patient_contact = widgets.Text(description=\"Contact:\")\n",
    "\n",
    "treatment_desc = widgets.Text(description=\"Treatment:\")\n",
    "treatment_cost = widgets.FloatText(description=\"Cost:\")\n",
    "\n",
    "treatments = []\n",
    "\n",
    "def add_patient_and_treatment(name, age, gender, contact, desc, cost):\n",
    "    global patient_id\n",
    "    patient_id = hbs.add_patient(name, age, gender, contact)\n",
    "    treatments.append({\"description\": desc, \"cost\": cost})\n",
    "    print(f\"Patient added. Treatment recorded. Total treatments: {len(treatments)}\")\n",
    "\n",
    "def generate_bill(b):\n",
    "    bill_id = hbs.create_bill(patient_id, treatments)\n",
    "    hbs.display_bill(bill_id)\n",
    "    hbs.export_to_excel(bill_id, f\"bill_{bill_id}.xlsx\")\n",
    "\n",
    "add_button = widgets.Button(description=\"Add Patient & Treatment\")\n",
    "add_button.on_click(lambda b: add_patient_and_treatment(\n",
    "    patient_name.value, patient_age.value, patient_gender.value, \n",
    "    patient_contact.value, treatment_desc.value, treatment_cost.value\n",
    "))\n",
    "\n",
    "bill_button = widgets.Button(description=\"Generate Bill\")\n",
    "bill_button.on_click(generate_bill)\n",
    "\n",
    "display(widgets.VBox([\n",
    "    widgets.HTML(\"<h2>Patient Information</h2>\"),\n",
    "    patient_name, patient_age, patient_gender, patient_contact,\n",
    "    widgets.HTML(\"<h2>Treatment Information</h2>\"),\n",
    "    treatment_desc, treatment_cost,\n",
    "    add_button, bill_button\n",
    "]))"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

NameError: name 'null' is not defined